In [28]:
!pip install allennlp
!pip install allennlp-models
!pip install urllib3 --upgrade
!pip install vaderSentiment
!pip install allennlp.predictors.predictor
!pip install --upgrade google-cloud-storage
!pip install checklist
!python -m spacy download en_core_web_lg

  Using cached torch-1.10.0-cp37-cp37m-manylinux1_x86_64.whl (881.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1


  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
firebase-admin 4.4.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0; platform_python_implementation != "PyPy", but you have google-api-core 2.7.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 4.4.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0; platform_python_implementation != "PyPy", but you have google-api-core 2.7.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


ERROR: Could not find a version that satisfies the requirement allennlp.predictors.predictor (from versions: none)
ERROR: No matching distribution found for allennlp.predictors.predictor
     |████████████████████████████████| 827.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [29]:
! git clone https://github.com/cpow24/long-doc-coref.git
! pip install torch==1.8.1
! pip install transformers==4.2.2
! pip install scipy==1.4.1

fatal: destination path 'long-doc-coref' already exists and is not an empty directory.
  Using cached torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (804.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.


In [1]:
import pandas as pd
import spacy
import nltk
import re
import networkx as nx
import numpy as np
import itertools
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from google.colab import drive 
import sys
sys.path.append('long-doc-coref/src')

drive.mount('/content/gdrive')
nltk.download('punkt')
nltk.download('omw-1.4')

from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import tqdm
import json

# This will also download the SpanBERT model finetuned for Coreference (by Joshi et al, 2020) from Huggingface
from inference.inference import Inference
from  inference.tokenize_doc import *
from transformers import BertTokenizerFast
from  inference.tokenize_doc import *

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
inference_model = Inference("gdrive/My Drive/Colab_Notebooks/Coref_Project/model.pth")

{'base_data_dir': '/share/data/speech/shtoshni/research/litbank_coref/data', 'base_model_dir': '/share/data/speech/shtoshni/research/litbank_coref/models', 'dataset': 'litbank', 'conll_scorer': '../lrec2020-coref/reference-coreference-scorers/scorer.pl', 'model_size': 'large', 'doc_enc': 'overlap', 'pretrained_bert_dir': '/share/data/speech/shtoshni/resources', 'max_segment_len': 512, 'max_span_width': 20, 'ment_emb': 'attn', 'top_span_ratio': 0.3, 'mem_type': 'learned', 'num_cells': 20, 'mlp_size': 3000, 'mlp_depth': 1, 'entity_rep': 'wt_avg', 'emb_size': 20, 'cross_val_split': 3, 'new_ent_wt': 2.0, 'num_train_docs': None, 'max_training_segments': 5, 'sample_invalid': 0.25, 'dropout_rate': 0.3, 'label_smoothing_wt': 0.0, 'max_epochs': 25, 'seed': 0, 'init_lr': 0.0002, 'no_singletons': False, 'eval': False, 'slurm_id': '6077327_172', 'model_dir': '/share/data/speech/shtoshni/research/litbank_coref/models/coref_aff65ce80c7eefcce3c2451b554e1e68', 'best_model_dir': '/share/data/speech/sht

In [3]:
# import Illiad as text file
with open('gdrive/My Drive/Colab_Notebooks/Coref_Project/Iliad.txt') as f:
  doc = f.read()
doc = re.sub('\n', ' ', doc)

In [4]:
#extract book 1 from the Illiad
re.finditer('BOOK I', doc)
[(m.start(0), m.end(0)) for m in re.finditer('BOOK I', doc)]
book_1 = doc[16:2000]

#tokenize book 1
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
#tokens, subtoken_map = get_tokenized_doc(book_1, tokenizer)
tokens = get_tokenized_doc(book_1, tokenizer)
doc_tokens = flatten(tokens['sentences'])

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


In [5]:
spacy.load('en_core_web_lg')

In [6]:
import en_core_web_lg

In [7]:
nlp = en_core_web_lg.load()
nlp.max_length = 5000000

In [8]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

In [9]:
#Named entity extraction
ner = nlp(book_1)
entity = []
start_idx = []
end_idx = []
ent_type = []

for ent in ner.ents:
  entity.append(ent.text)
  ent_type.append(ent.label_)
  start_idx.append(ent.start_char)
  end_idx.append(ent.end_char)

In [10]:
ner_df = pd.DataFrame(list(zip(entity, ent_type, start_idx, end_idx )),
                      columns=['entity', 'entity_type', 'start_idx', 'end_idx'])

In [11]:
ner_df

,entity,entity_type,start_idx,end_idx
0,ACHILLES,PERSON,29,37
1,Troy,PERSON,69,73
2,Greeks,NORP,79,85
3,two,CARDINAL,154,157
4,Chryseïs,ORG,178,186
5,Briseïs,ORG,191,198
6,first,ORDINAL,213,218
7,Agamemnon,PERSON,222,231
8,Achilles,PERSON,249,257
9,Chryses,PERSON,259,266


In [12]:
#Capitalize only first letter of each name
ner_df['char'] = ner_df['entity'].apply(lambda x: x.capitalize())

In [13]:
#Top 20 most frequent characters mentioned
char_list = list(ner_df[ner_df['entity_type'] == 'PERSON']['char'].value_counts()[:20].index)

In [14]:
#Resolve issue of 'PERSON' entities wrongly identified as non-person identities
char_df = ner_df[ner_df['entity'].isin(char_list)]

In [15]:
"""
Inverse mapping of tokenized text to list text
"""
def inv_map(tokenized_text):
  text = []
  for i in tokenized_text:
    if len(i) < 3:
      text.append(i)
    elif i[0:2] != '##':
      text.append(i)
    else:
      text[-1] = text[-1] + i[2:]
  return text

In [16]:
token_doc = inv_map(doc_tokens)

In [17]:
print(token_doc)

['BOOK', 'I', '.', 'THE', 'CONTENTION', 'OF', 'ACHILLES', 'AND', 'AGAMEMNON', '.', 'In', 'the', 'war', 'of', 'Troy', ',', 'the', 'Greeks', 'having', 'sacked', 'some', 'of', 'the', 'neighbouring', 'towns', ',', 'and', 'taken', 'from', 'thence', 'two', 'beautiful', 'captives', ',', 'Chryseïs', 'and', 'Briseïs', ',', 'allotted', 'the', 'first', 'to', 'Agamemnon', ',', 'and', 'the', 'last', 'to', 'Achilles', '.', 'Chryses', ',', 'the', 'father', 'of', 'Chryseïs', ',', 'and', 'priest', 'of', 'Apollo', ',', 'comes', 'to', 'the', 'Grecian', 'camp', 'to', 'ransom', 'her', ';', 'with', 'which', 'the', 'action', 'of', 'the', 'poem', 'opens', ',', 'in', 'the', 'tenth', 'year', 'of', 'the', 'siege', '.', 'The', 'priest', 'being', 'refused', ',', 'and', 'insolently', 'dismissed', 'by', 'Agamemnon', ',', 'entreats', 'for', 'vengeance', 'from', 'his', 'god', ';', 'who', 'inflicts', 'a', 'pestilence', 'on', 'the', 'Greeks', '.', 'Achilles', 'calls', 'a', 'council', ',', 'and', 'encourages', 'Chalcas',

In [18]:
char_list

['Achilles',
 'Agamemnon',
 'Thetis',
 'Troy',
 'Chryses',
 'Chalcas',
 'Nestor',
 'Juno',
 'Chrysa',
 'Atrides']

In [19]:
'''
Given a list of characters and a tokenized document, this function returns a
dictionary of dictionaries containing a unique character ID and the occurence 
indices for each character in the tokenized document.
'''
def get_char_occurrences(char_list, tokenized_doc):
  #Dictionary of character occurrence indices
  char_occ_dict = {}

  for id, char in enumerate(char_list):
    char_occ_dict[id] = {'name': char, 'occ':[i for i, j in enumerate(tokenized_doc) if j == char]}

  return char_occ_dict

In [20]:
char_occ = get_char_occurrences(char_list, token_doc)

In [21]:
char_occ

{0: {'name': 'Achilles', 'occ': [48, 114, 155, 180, 316, 376]},
 1: {'name': 'Agamemnon', 'occ': [42, 97]},
 2: {'name': 'Thetis', 'occ': [198, 293]},
 3: {'name': 'Troy', 'occ': [14]},
 4: {'name': 'Chryses', 'occ': [50]},
 5: {'name': 'Chalcas', 'occ': [121]},
 6: {'name': 'Nestor', 'occ': [158]},
 7: {'name': 'Juno', 'occ': [229]},
 8: {'name': 'Chrysa', 'occ': [309]},
 9: {'name': 'Atrides', 'occ': [378]}}